## ⚠️ Security Notice

API keys and secrets have been sanitized from this notebook for security.
Please replace placeholder values with your actual API keys before running.


In [ ]:
!pip install faiss-cpu sentence-transformers beautifulsoup4 lxml PyMuPDF


In [ ]:
!pip install pandas
!pip install xbrl
!pip install beautifulsoup4 lxml
!pip install pymupdf
!pip install langchain huggingface_hub
!pip install -U langchain-huggingface

In [ ]:
!pip install openai
!pip install -U openai langchain

In [ ]:
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-YOUR_OPENAI_PROJECT_KEY_HERE"


In [ ]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2
)


In [ ]:
# Core imports
import os
import fitz  # PyMuPDF
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
def parse_pdf(file_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def parse_html(file_path):
    """Extracts text from an HTML/HTM file."""
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "lxml")
        return soup.get_text(separator="\n")

def parse_xml(file_path):
    """Extracts text from an XML file."""
    tree = ET.parse(file_path)
    root = tree.getroot()
    text = ET.tostring(root, encoding="unicode", method="text")
    return text

In [ ]:
def chunk_text(text, chunk_size=500, overlap=50):
    """
    Splits text into overlapping chunks.
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

In [ ]:
def embed_chunks(chunks):
    """
    Converts text chunks to embeddings using sentence-transformers.
    """
    embeddings = embedder.encode(
        chunks,
        convert_to_numpy=True,
        show_progress_bar=True
    )
    return embeddings

In [ ]:
from google.colab import files

# Upload multiple files
uploaded_files = files.upload()

# Map file extensions to parser functions
parser_map = {
    ".pdf": parse_pdf,
    ".htm": parse_html,
    ".html": parse_html,
    ".xml": parse_xml
}

# Helper to choose parser based on extension
def get_parser(file_name):
    ext = os.path.splitext(file_name)[1].lower()
    return parser_map.get(ext, None)

In [ ]:
all_chunks = []
all_embeddings = []
all_metadata = []

for file_name in uploaded_files.keys():
    parser = get_parser(file_name)
    if parser is None:
        print(f"⚠️ Skipping unsupported file: {file_name}")
        continue

    print(f"✅ Processing: {file_name}")

    # Parse text
    text = parser(file_name)

    # Chunk text
    chunks = chunk_text(text, chunk_size=500, overlap=50)

    # Embed chunks
    embeddings = embed_chunks(chunks)

    # Collect
    all_chunks.extend(chunks)
    all_embeddings.append(embeddings)

    # Store metadata for each chunk
    all_metadata.extend([
        {"file_name": file_name, "chunk_id": idx}
        for idx in range(len(chunks))
    ])

print("✅ All files processed.")


In [ ]:
# Stack all embeddings into one big matrix
embedding_matrix = np.vstack(all_embeddings)

# Create FAISS index
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embedding_matrix)

print(f"✅ FAISS index created with {index.ntotal} vectors.")


In [ ]:
def semantic_search(query, k=5):
    """
    Embeds the query and retrieves top-k most similar chunks with metadata.
    """
    # Embed the query
    q_embedding = embedder.encode([query])

    # Search FAISS
    distances, indices = index.search(np.array(q_embedding), k)

    # Collect results
    results = []
    for idx in indices[0]:
        result = {
            "text": all_chunks[idx],
            "metadata": all_metadata[idx]
        }
        results.append(result)

    return results


In [ ]:
def display_results(results):
    """
    Pretty-print retrieved chunks with metadata.
    """
    for i, r in enumerate(results, start=1):
        print(f"\n🔹 Result #{i}")
        print(f"📄 File: {r['metadata']['file_name']} | Chunk ID: {r['metadata']['chunk_id']}")
        print("------")
        snippet = r["text"][:500]  # Show only first 500 characters
        print(snippet)


In [ ]:
# Example query
query = "What are Apple's supply chain risks?"

# Run search
results = semantic_search(query, k=5)

# Display results
display_results(results)


In [ ]:
from langchain_openai import ChatOpenAI

# Initialize your LLM (change model if needed)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2
)

def summarize_with_llm(query, results):
    """
    Combines retrieved chunks and sends them to the LLM to generate an answer.
    """
    # Build context with citations
    context = ""
    for r in results:
        snippet = r["text"][:1000].strip().replace("\n", " ")
        file = r["metadata"]["file_name"]
        context += f"\n(Source: {file})\n{snippet}\n"

    # Prompt
    prompt = (
        f"You are a financial analyst assistant.\n\n"
        f"Context excerpts:\n{context}\n\n"
        f"Question: {query}\n\n"
        f"Answer in bullet points with clear sentences."
    )

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
# Example question
query = "Summarize Apple's supply chain risks."

# Retrieve top 5 chunks
results = semantic_search(query, k=5)

# Generate answer
summary = summarize_with_llm(query, results)

# Display answer
print("✅ LLM Summary:\n")
print(summary)


In [ ]:
# Example 2
query2 = "What were the key financial highlights in 2023?"
results2 = semantic_search(query2, k=5)
summary2 = summarize_with_llm(query2, results2)
print("✅ Financial Highlights Summary:\n")
print(summary2)


In [ ]:
from lxml import etree

def extract_xbrl_facts(xml_file):
    """
    Extracts all us-gaap facts and their values from an XBRL XML file.
    Returns a list of dicts with label and value.
    """
    tree = etree.parse(xml_file)
    root = tree.getroot()

    ns = {"us-gaap": "http://fasb.org/us-gaap/2023"}

    # Find all tags in the us-gaap namespace
    facts = []
    for element in root.iter():
        if element.tag.startswith("{http://fasb.org/us-gaap/2023}"):
            tag = element.tag.split("}")[1]
            value = element.text
            facts.append({
                "Label": tag,
                "Value": value
            })
    return facts


In [ ]:
# Path to your uploaded XBRL file
xbrl_file = "aapl-20230930_htm.xml"

# Extract all facts
facts = extract_xbrl_facts(xbrl_file)

# Show a sample of what we got
print(f"✅ Extracted {len(facts)} facts. Showing first 10:\n")
for f in facts[:10]:
    print(f)


In [ ]:
# Define the tags we care about
important_tags = {
    "RevenueFromContractWithCustomerExcludingAssessedTax": "Revenue",
    "Revenues": "Revenue",
    "SalesRevenueNet": "Revenue",
    "CostOfGoodsAndServicesSold": "COGS",
    "OperatingIncomeLoss": "OperatingIncome",
    "NetIncomeLoss": "NetIncome",
    "Assets": "TotalAssets",
    "Liabilities": "TotalLiabilities",
    "StockholdersEquity": "Equity",
    "LongTermDebt": "LongTermDebt",
}

# Initialize empty dict
data = {}

# Loop through extracted facts
for f in facts:
    tag = f["Label"]
    value = f["Value"]
    if tag in important_tags:
        try:
            # Convert to numeric safely
            numeric_value = float(value)
            data[important_tags[tag]] = numeric_value
        except:
            continue

# Display cleaned data
print("✅ Extracted Financial Metrics:\n")
for k, v in data.items():
    print(f"{k}: {v:,.0f}")


In [ ]:
import pandas as pd

# Convert dict to DataFrame
df = pd.DataFrame([data])

# Compute Ratios
df["GrossMargin"] = (df["Revenue"] - df["COGS"]) / df["Revenue"] * 100
df["OperatingMargin"] = df["OperatingIncome"] / df["Revenue"] * 100
df["NetMargin"] = df["NetIncome"] / df["Revenue"] * 100
df["DebtEquity"] = df["LongTermDebt"] / df["Equity"]
df["LiabilitiesEquity"] = df["TotalLiabilities"] / df["Equity"]

# Display nicely
print("✅ Financial Ratios:\n")
print(df[[
    "GrossMargin",
    "OperatingMargin",
    "NetMargin",
    "DebtEquity",
    "LiabilitiesEquity"
]].round(2))


In [ ]:
import matplotlib.pyplot as plt

# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Bar plot of the ratios
bars = ax.bar(
    ["Gross Margin", "Operating Margin", "Net Margin"],
    [
        df["GrossMargin"].iloc[0],
        df["OperatingMargin"].iloc[0],
        df["NetMargin"].iloc[0]
    ],
    color="#5fba7d"
)

# Add value labels
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.5, f"{yval:.2f}%", ha='center', fontsize=12)

# Customize plot
ax.set_title("Apple Financial Margins (%)", fontsize=16)
ax.set_ylabel("Percentage")
ax.set_ylim(0, max(df["GrossMargin"].iloc[0], df["OperatingMargin"].iloc[0], df["NetMargin"].iloc[0]) + 10)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
from bs4 import BeautifulSoup

file = "/content/aapl-20210925_htm.xml"

with open(file, "r") as f:
    soup = BeautifulSoup(f, "lxml")

# List unique tag names
all_tags = set([tag.name for tag in soup.find_all()])

print("✅ Unique tag names in 2021 file:")
for t in sorted(all_tags):
    print(t)


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files


In [ ]:
uploaded = files.upload()


In [ ]:
import glob

# Only pick Apple filings ending with _htm.xml
files = sorted(glob.glob("aapl-*_htm.xml"))

print(f"✅ Found {len(files)} files:")
for f in files:
    print(" -", f)


In [ ]:
from bs4 import BeautifulSoup
import glob

# Collect only aapl-*_htm.xml files in current directory
xml_files = sorted(glob.glob("aapl-*_htm.xml"))

if not xml_files:
    print("⚠️ No matching files found.")
else:
    print(f"✅ Found {len(xml_files)} files:")
    for f in xml_files:
        print(f" - {f}")

# Dictionary to hold all tags per file
all_file_tags = {}

# Process each file
for file in xml_files:
    with open(file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "xml")

    # Get all unique tag names
    tags = sorted({tag.name for tag in soup.find_all()})
    all_file_tags[file] = tags

# Display all tags found in each file
for file, tags in all_file_tags.items():
    print(f"\n✅ {file} has {len(tags)} unique tags:")
    print(tags)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import glob

# Prepare list of tags we care about
kpi_tags = {
    "Revenue": "RevenueFromContractWithCustomerExcludingAssessedTax",
    "NetIncome": "NetIncomeLoss",
    "Assets": "Assets",
    "Liabilities": "Liabilities",
    "Equity": "StockholdersEquity",
    "LongTermDebt": "LongTermDebtNoncurrent"
}

# Collect only aapl-*_htm.xml files
xml_files = sorted(glob.glob("aapl-*_htm.xml"))

if not xml_files:
    print("⚠️ No matching files found.")
else:
    print(f"✅ Found {len(xml_files)} files to process:")
    for f in xml_files:
        print(f" - {f}")

# List to hold data rows
data = []

# Process each file
for file in xml_files:
    with open(file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "xml")

    row = {"File": file}
    for kpi, tag in kpi_tags.items():
        element = soup.find(tag)
        value = element.text.strip() if element and element.text else None
        row[kpi] = value
    data.append(row)

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import glob

# Prepare list of tags we care about
kpi_tags = {
    "Revenue": "RevenueFromContractWithCustomerExcludingAssessedTax",
    "NetIncome": "NetIncomeLoss",
    "Assets": "Assets",
    "Liabilities": "Liabilities",
    "Equity": "StockholdersEquity",
    "LongTermDebt": "LongTermDebtNoncurrent",
    "FiscalYear": "DocumentFiscalYearFocus"
}


# Collect only aapl-*_htm.xml files
xml_files = sorted(glob.glob("aapl-*_htm.xml"))
data = []

for file in xml_files:
    with open(file, "r") as f:
        soup = BeautifulSoup(f, "xml")

    row = {"File": file}
    for kpi, tag in kpi_tags.items():
        element = soup.find(tag)
        if element:
            raw_value = element.text.strip()
            clean_value = raw_value.replace(",", "").replace("$", "")
            try:
                numeric_value = float(clean_value)
            except ValueError:
                numeric_value = None
        else:
            numeric_value = None
        row[kpi] = numeric_value
    data.append(row)

df = pd.DataFrame(data)
df


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Convert KPI columns to numeric
kpi_cols = ["Revenue", "NetIncome", "Assets", "Liabilities", "Equity"]
for col in kpi_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Extract year
df["Year"] = df["File"].str.extract(r"(\d{4})").astype(int)
df_sorted = df.sort_values("Year")

# Create the figure
fig = go.Figure()

# Custom color palette
colors = ["#1f77b4", "#2ca02c", "#d62728"]

# Add one bar trace per year
for i, (_, row) in enumerate(df_sorted.iterrows()):
    fig.add_trace(
        go.Bar(
            x=kpi_cols,
            y=[row[kpi] for kpi in kpi_cols],
            name=str(row["Year"]),
            marker_color=colors[i]
        )
    )

# Update layout for clarity
fig.update_layout(
    title="Apple Financial KPIs by Year",
    xaxis_title="KPI",
    yaxis_title="Amount (USD Millions)",
    barmode="group",
    legend_title="Fiscal Year",
    template="plotly_white",
    font=dict(size=13)
)

fig.show()
